In [ ]:
#%cd /content/drive/MyDrive/SAR Images/


In [ ]:
# Unzip the first file
#!unzip SAR_patch.zip -d /content/drive/MyDrive/bw_one

# Unzip the second file
#!unzip patch.zip -d /content/drive/MyDrive/color_one


In [ ]:
#!unzip SAR_patch.zip -d /content/drive/MyDrive/bw_one

In [ ]:
import os
import tensorflow as tf

# Paths to extracted images
dir_A = '/content/drive/MyDrive/bw_one/SAR_patchnew'
dir_B = '/content/drive/MyDrive/color_one/patch'

# Function to pair images and load them as datasets
def load_image_pair(image_name, dir_A, dir_B):
    # Convert TensorFlow string tensor to a Python string
    image_name_str = image_name.numpy().decode('utf-8')

    img_A = tf.io.read_file(os.path.join(dir_A, image_name_str))
    img_A = tf.image.decode_tiff(img_A)

    img_B = tf.io.read_file(os.path.join(dir_B, image_name_str))
    img_B = tf.image.decode_tiff(img_B)

    # Resize or preprocess images as needed
    img_A = tf.image.resize(img_A, [256, 256])
    img_B = tf.image.resize(img_B, [256, 256])

    return img_A, img_B

# Wrap the function to handle tensors using tf.py_function
def load_image_pair_wrapper(image_name):
    return tf.py_function(load_image_pair, [image_name, dir_A, dir_B], [tf.float32, tf.float32])

# Load dataset
image_names = os.listdir(dir_A)
dataset = tf.data.Dataset.from_tensor_slices(image_names)

# Apply the function to each element in the dataset
dataset = dataset.map(load_image_pair_wrapper)

# Batch and prefetch
batch_size = 32
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

# Function to display a pair of images (original and target)
def show_images(image_A, image_B, title_A='Input Image', title_B='Target Image'):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title(title_A)
    plt.imshow(tf.squeeze(image_A), cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title(title_B)
    plt.imshow(tf.squeeze(image_B), cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [ ]:
import os
import tensorflow as tf

# Paths to extracted images
dir_A = '/content/drive/MyDrive/bw_one/SAR_patchnew'
dir_B = '/content/drive/MyDrive/color_one/patch'

# Function to pair images and load them as datasets
def load_image_pair(image_name, dir_A, dir_B):
    # Convert TensorFlow string tensor to a Python string
    image_name_str = image_name.numpy().decode('utf-8')

    # Use image_name_str which is a string instead of image_name which is a Tensor
    img_A = tf.io.read_file(os.path.join(dir_A, image_name_str))
    img_A = tf.image.decode_tiff(img_A)

    # Use image_name_str which is a string instead of image_name which is a Tensor
    img_B = tf.io.read_file(os.path.join(dir_B, image_name_str))
    img_B = tf.image.decode_tiff(img_B)

    # Resize or preprocess images as needed
    img_A = tf.image.resize(img_A, [256, 256])
    img_B = tf.image.resize(img_B, [256, 256])

    return img_A, img_B

# Wrap the function to handle tensors using tf.py_function
def load_image_pair_wrapper(image_name):
    return tf.py_function(load_image_pair, [image_name, dir_A, dir_B], [tf.float32, tf.float32])

# Load dataset
image_names = os.listdir(dir_A)
dataset = tf.data.Dataset.from_tensor_slices(image_names)

# Apply the function to each element in the dataset
dataset = dataset.map(load_image_pair_wrapper)

# Batch and prefetch
batch_size = 32
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [ ]:
import os
import cv2
import numpy as np
sar_dir = '/content/drive/MyDrive/bw_one/SAR_patchnew'
optical_dir = '/content/drive/MyDrive/color_one/patch'

def load_images_from_folders(sar_dir, optical_dir):
    sar_images = []
    optical_images = []

    sar_files = sorted(os.listdir(sar_dir))
    optical_files = sorted(os.listdir(optical_dir))

    # Debugging: Print the list of files
    print(f"SAR files: {sar_files}")
    print(f"Optical files: {optical_files}")

    # Create dictionaries to map full filenames
    sar_dict = {os.path.splitext(f)[0]: f for f in sar_files}  # Extract filename without extension
    optical_dict = {os.path.splitext(f)[0]: f for f in optical_files}

    for key in sar_dict:
        if key in optical_dict:
            sar_file = sar_dict[key]
            optical_file = optical_dict[key]

            # Debugging: Print the matched filenames
            print(f"Matching {sar_file} with {optical_file}")

            sar_img = cv2.imread(os.path.join(sar_dir, sar_file), cv2.IMREAD_GRAYSCALE)
            optical_img = cv2.imread(os.path.join(optical_dir, optical_file))

            if sar_img is not None and optical_img is not None:
                # Normalize images to [0, 1]
                sar_img = sar_img.astype(np.float32) / 255.0
                optical_img = optical_img.astype(np.float32) / 255.0

                # Add channel dimension for grayscale SAR image
                sar_img = np.expand_dims(sar_img, axis=-1)

                sar_images.append(sar_img)
                optical_images.append(optical_img)
            else:
                print(f"Warning: Could not load {sar_file} or {optical_file}")
        else:
            print(f"Warning: Could not find match for: {key}")

    return np.array(sar_images), np.array(optical_images)

# Example usage:
# sar_dir = 'archive/v_2/urban/s1'
# optical_dir = 'archive/v_2/urban/s2'
# sar_images, optical_images = load_images_from_folders(sar_dir, optical_dir)
# print(f"Loaded {len(sar_images)} SAR images and {len(optical_images)} optical images.")


In [ ]:
sar_images, optical_images = load_images_from_folders(sar_dir, optical_dir)

SAR files: ['patch_0_x89_y15.tif', 'patch_1000_x85_y30.tif', 'patch_1001_x86_y30.tif', 'patch_1002_x87_y30.tif', 'patch_1003_x88_y30.tif', 'patch_1004_x89_y30.tif', 'patch_1005_x90_y30.tif', 'patch_1006_x91_y30.tif', 'patch_1007_x92_y30.tif', 'patch_1008_x93_y30.tif', 'patch_1009_x94_y30.tif', 'patch_100_x95_y17.tif', 'patch_1010_x95_y30.tif', 'patch_1011_x96_y30.tif', 'patch_1012_x97_y30.tif', 'patch_1013_x98_y30.tif', 'patch_1014_x99_y30.tif', 'patch_1015_x100_y30.tif', 'patch_1016_x101_y30.tif', 'patch_1017_x102_y30.tif', 'patch_1018_x33_y31.tif', 'patch_1019_x34_y31.tif', 'patch_101_x96_y17.tif', 'patch_1020_x35_y31.tif', 'patch_1021_x36_y31.tif', 'patch_1022_x37_y31.tif', 'patch_1023_x38_y31.tif', 'patch_1024_x39_y31.tif', 'patch_1025_x40_y31.tif', 'patch_1026_x41_y31.tif', 'patch_1027_x42_y31.tif', 'patch_1028_x43_y31.tif', 'patch_1029_x44_y31.tif', 'patch_102_x97_y17.tif', 'patch_1030_x45_y31.tif', 'patch_1031_x46_y31.tif', 'patch_1032_x47_y31.tif', 'patch_1033_x48_y31.tif', 'pa